In [1]:
'''Import libraries'''
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification, AdamW, Trainer, TrainingArguments
from tqdm import tqdm
from torch.nn import functional as F
import torch.nn as nn

wandb.login()

c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alberto-rodero557. Use `wandb login --relogin` to force relogin


True

In [2]:
'''Variables and parameters'''

SAMPLES_TO_TRAIN=-1
DIMENSIONS=200

N_LABELS=2
MAX_LEN = 256
EPOCHS=100
PATIENCE=10
LEARNING_RATE=.00005
WEIGHT_DECAY=.01
BATCH_SIZE=16
METRIC_FOR_BEST_MODEL='eval_loss'
if METRIC_FOR_BEST_MODEL=='eval_loss':
    GREATER_IS_BETTER = False
else:
    GREATER_IS_BETTER = True

In [3]:
'''Preparing dataset'''
import pickle

file_path = 'datasets/subtaskA_glove_train_dev_monolingual.pkl'

# Load the data from the pickle file
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Extract the individual datasets from the loaded data
train_x = data['train_x']
train_y = data['train_y']
dev_x = data['dev_x']
dev_y = data['dev_y']

if SAMPLES_TO_TRAIN!=-1:
    random_indices = np.random.choice(train_x.shape[0], size=SAMPLES_TO_TRAIN, replace=False)
    train_x = train_x[random_indices]
    train_y = train_y[random_indices]

    random_indices_dev = np.random.choice(dev_x.shape[0], size=int(train_x.shape[0] * 0.2), replace=False)
    dev_x = dev_x[random_indices_dev]
    dev_y = pd.Series(dev_y.to_numpy()[random_indices_dev])

print(train_x.shape)
print(train_y.shape)
print(dev_x.shape)
print(dev_y.shape)

print(type(train_y))
print(type(dev_y))

(119757, 200)
(119757,)
(5000, 200)
(5000,)
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [4]:
'''metrics'''

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'auc': auc,
        'precision': precision,
        'recall': recall,
    }

In [5]:
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)
        self.y = torch.tensor(y, dtype=torch.long)
        self.len = len(X)

    def __getitem__(self, index):
        return {'x': self.X[index], 'label': self.y[index], 'label_ids': index}

    def __len__(self):
        return self.len

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y.values, test_size=0.2, random_state=42)
traindata = Data(X_train, y_train)
testdata = Data(X_test, y_test)

devdata=Data(dev_x, dev_y.values)

In [6]:
# number of features (len of X cols)
input_dim = train_x.shape[-1]

# number of classes (unique of y)
output_dim = 2

class CNN1D(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(CNN1D, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=100, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(100)
        self.conv2 = nn.Conv1d(in_channels=100, out_channels=150, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(150)
        self.dropout1 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(150 * 200, 256)
        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x, labels=None):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout1(x)
        
        # Flatten the output for the dense layer
        x = torch.flatten(x, 1) 
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(x, labels)
            return loss, x
        
        return x

# Instantiate the model with appropriate dimensions
model = CNN1D(input_dim=200, num_classes=2)

In [7]:
from transformers import EarlyStoppingCallback

model = CNN1D(input_dim=200, num_classes=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=500,
    weight_decay=WEIGHT_DECAY,
    metric_for_best_model=METRIC_FOR_BEST_MODEL,
    greater_is_better=GREATER_IS_BETTER,
    logging_dir='./logs',
    logging_steps=15000,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=False,
    logging_first_step=False,
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="wandb"
)

# Create trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=traindata,             # training dataset
    eval_dataset=testdata, 
    compute_metrics=compute_metrics,# training dataset
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)]
)

# Train the model
print(trainer.evaluate())

trainer.train()

print(trainer.evaluate())

 99%|█████████▉| 1479/1497 [00:01<00:00, 1209.74it/s]

100%|██████████| 1497/1497 [00:01<00:00, 789.28it/s] 


{'eval_loss': 0.6937636137008667, 'eval_accuracy': 0.47828991315965264, 'eval_f1': 0.6470854044283778, 'eval_auc': 0.5, 'eval_precision': 0.47828991315965264, 'eval_recall': 1.0, 'eval_runtime': 3.0591, 'eval_samples_per_second': 7829.627, 'eval_steps_per_second': 489.352}


                                                      
  1%|          | 6010/598800 [00:20<3:13:58, 50.94it/s]

{'eval_loss': 0.45240145921707153, 'eval_accuracy': 0.7842768871075484, 'eval_f1': 0.7459060732726825, 'eval_auc': 0.7791890052861609, 'eval_precision': 0.8541502421443856, 'eval_recall': 0.6620111731843575, 'eval_runtime': 1.5479, 'eval_samples_per_second': 15473.835, 'eval_steps_per_second': 967.115, 'epoch': 1.0}


...

{'loss': 0.3051, 'learning_rate': 2.246364699983286e-05, 'epoch': 55.11}


                                                        
 56%|█████▌    | 335328/598800 [18:43<14:42, 298.55it/s]


{'eval_loss': 0.31642377376556396, 'eval_accuracy': 0.8662324649298597, 'eval_f1': 0.8592267135325132, 'eval_auc': 0.8657037291039279, 'eval_precision': 0.8650035385704176, 'eval_recall': 0.8535265363128491, 'eval_runtime': 1.2752, 'eval_samples_per_second': 18783.159, 'eval_steps_per_second': 1173.947, 'epoch': 56.0}
{'train_runtime': 1123.1755, 'train_samples_per_second': 8529.834, 'train_steps_per_second': 533.131, 'train_loss': 0.35676768882002285, 'epoch': 56.0}


100%|██████████| 1497/1497 [00:01<00:00, 1141.13it/s]

{'eval_loss': 0.31528955698013306, 'eval_accuracy': 0.864562458249833, 'eval_f1': 0.8550621034760075, 'eval_auc': 0.8633440354544739, 'eval_precision': 0.8758008420281896, 'eval_recall': 0.8352828212290503, 'eval_runtime': 1.3139, 'eval_samples_per_second': 18230.29, 'eval_steps_per_second': 1139.393, 'epoch': 56.0}


In [8]:
trainer.evaluate(devdata)

100%|██████████| 313/313 [00:00<00:00, 1103.29it/s]


{'eval_loss': 1.1675323247909546,
 'eval_accuracy': 0.5634,
 'eval_f1': 0.3821115199547127,
 'eval_auc': 0.5634,
 'eval_precision': 0.6534365924491772,
 'eval_recall': 0.27,
 'eval_runtime': 0.2857,
 'eval_samples_per_second': 17501.089,
 'eval_steps_per_second': 1095.568,
 'epoch': 56.0}

In [9]:
trainer.save_model('SavedModels/'+'cnn_'+str(round(SAMPLES_TO_TRAIN/1000))+'k')

In [11]:
# All samples trained with training dataset
print(''''eval_loss': 0.31528955698013306, 'eval_accuracy': 0.864562458249833, 'eval_f1': 0.8550621034760075, 'eval_auc': 0.8633440354544739, 'eval_precision': 0.8758008420281896, 'eval_recall': 0.8352828212290503''')

'eval_loss': 0.31528955698013306, 'eval_accuracy': 0.864562458249833, 'eval_f1': 0.8550621034760075, 'eval_auc': 0.8633440354544739, 'eval_precision': 0.8758008420281896, 'eval_recall': 0.8352828212290503
